In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import json
import tensorflow.keras.layers as L
import keras.backend as K
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.cluster import KMeans
import os

# Data Augmentation

In [ ]:
!wget https://www.tbi.univie.ac.at/RNA/download/ubuntu/ubuntu_18_04/viennarna_2.4.15-1_amd64.deb
!apt-get install ./viennarna_2.4.15-1_amd64.deb -y
!git clone https://github.com/DasLab/arnie

!/opt/conda/bin/python3.7 -m pip install --upgrade pip
!git clone https://www.github.com/DasLab/draw_rna draw_rna_pkg
!cd draw_rna_pkg && python setup.py install

!yes '' | cpan -i Graph
!git clone https://github.com/hendrixlab/bpRNA

In [ ]:
import sys

!echo "vienna_2: /usr/bin" > arnie.conf
!echo "TMP: /kaggle/working/tmp" >> arnie.conf
!mkdir -p /kaggle/working/tmp
os.environ["ARNIEFILE"] = f"/kaggle/working/arnie.conf"
sys.path.append('/kaggle/working/draw_rna_pkg/')
sys.path.append('/kaggle/working/draw_rna_pkg/ipynb/')
pkg = 'vienna_2'

In [ ]:
from multiprocessing import Pool
from arnie.pfunc import pfunc
from arnie.mea.mea import MEA
from arnie.free_energy import free_energy
from arnie.bpps import bpps
from arnie.mfe import mfe
import arnie.utils as utils
from tqdm.notebook import tqdm as tqdm

n_candidates = 2
# turn off for all data
debug = True

In [ ]:
!grep processor /proc/cpuinfo | wc -l

MAX_THRE = 4


In [ ]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
if debug:
    train = train[:20]
    test = test[:20]
target_df = train.append(test)

# Start

In [ ]:
def proc1(arg):
    sequence = arg[0]
    id = arg[1]
    log_gamma = arg[2]
    bp_matrix = bpps(sequence, package=pkg)
    mea_mdl = MEA(bp_matrix,gamma=10**log_gamma)
    return id, sequence, mea_mdl.structure, log_gamma, mea_mdl.score_expected()[2]

li = []
for log_gamma in range(10):
    for i, arr in enumerate(target_df[['sequence','id']].values):
        li.append([arr[0], arr[1], log_gamma])

p = Pool(processes=MAX_THRE)
results = []
for ret in tqdm(p.imap(proc1, li),total=len(li)):
    results.append(ret)
    #print(f'done for {ret[0]}')
df = pd.DataFrame(results, columns=['id', 'sequence', 'structure', 'log_gamma', 'score'])

df_tmp = target_df[['id', 'sequence', 'structure']].copy()
df_tmp['log_gamma'] = 100
df_tmp['score'] = 100
df = df.append(df_tmp).sort_values('score', ascending=False).reset_index(drop=True)

new_df = pd.DataFrame()
for id in df['id'].unique():
    unq_df = df[df['id'] == id].drop_duplicates('structure')
    unq_df['cnt'] = unq_df.shape[0]
    new_df = new_df.append(unq_df[1:min(n_candidates,len(unq_df))])

In [ ]:
!mkdir -p tmp_files
def get_predicted_loop_type(id, sequence, structure, debug=False):
    structure_fixed = structure.replace('.','0').replace('(','1').replace(')','2')
    pid = os.getpid()
    tmp_in_file = f'tmp_files/{id}_{structure_fixed}_{pid}.dbn'
    tmp_out_file = f'{id}_{structure_fixed}_{pid}.st'
    !echo $sequence > $tmp_in_file
    !echo "$structure" >> $tmp_in_file
    !export PERL5LIB=/root/perl5/lib/perl5 && perl bpRNA/bpRNA.pl $tmp_in_file
    result = [l.strip('\n') for l in open(tmp_out_file)]
    if debug:
        print(sequence)
        print(structure)
        print(result[5])
    else:
        !rm $tmp_out_file $tmp_in_file
    return id, structure, result[5]

def proc2(arg):
    result = get_predicted_loop_type(arg[0], arg[1], arg[2], debug=False)
    return result

li = []
for i, arr in enumerate(new_df[['id', 'sequence', 'structure']].values):
    li.append(arr)

p = Pool(processes=MAX_THRE)
results_loop_type = []
for ret in tqdm(p.imap(proc2, li),total=len(li)):
    results_loop_type.append(ret)
    #print(f'done for {ret[0]}')

new_df = new_df.merge(pd.DataFrame(results_loop_type, columns=('id', 'structure', 'predicted_loop_type')), on=['id','structure'], how='left')
new_df.to_csv('aug_data.csv', index=False)